In [104]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [105]:
# import raw data
df = pd.read_csv('../Data/Raw/ufo_raw.csv')
# confirm
df.head(1)

In [107]:
# copy into w(orking) df
wdf = df.copy()
# review columns
wdf.columns

Index(['datetime', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

In [108]:
# review dtypes and store them in dttype col
wdf['datetype'] = pd.to_datetime(wdf['datetime']).dt.date
# confirm
wdf.head(1)

,datetime,City,State,Shape,Duration,Summary,Posted,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,datetype
0,2006-01-01 01:20:00,I-80 (unknown city proximity),NY,Formation,0 days 00:00:20.000000000,"Traveling from Brooklyn, NY to Groton, NY on I...",6/7/19,NaN,NaN,NaN,NaN,2006-01-01
1,2006-01-01 23:00:00,San Antonio/Laredo (between),TX,Triangle,0 days 00:03:00.000000000,Three separate sightins which ocurred in rural...,6/9/09,NaN,NaN,NaN,NaN,2006-01-01
2,2006-01-02 00:00:00,Ann Arbor,MI,Changing,0 days 00:03:00.000000000,"Probable HOAX: 5 ufos, silent black objects, m...",10/30/06,NaN,NaN,NaN,NaN,2006-01-02
3,2006-01-02 00:00:00,Fredericksburg,VA,Light,0 days 00:02:00.000000000,clear sky looked like a star 10:00 am in the m...,2/1/07,NaN,NaN,NaN,NaN,2006-01-02
4,2006-01-04 07:45:00,Scotland,PA,Light,0 days 00:01:00.000000000,"Small, white, light in the northeast sky...dis...",2/1/07,NaN,NaN,NaN,NaN,2006-01-04


In [109]:
# add dttype col
wdf['dttyp'] = pd.to_datetime(wdf['datetype'])

In [110]:
# add week col (dtype=datetime)
wdf['weekofyear'] = wdf['dttyp'].dt.isocalendar().week

In [111]:
# add year col  (dtype=datetime)
wdf['year'] = wdf['dttyp'].dt.year

In [112]:
# add week_id col (dtype=object) 
wdf['year_week'] = wdf['year'].astype(str) + "-" + wdf['weekofyear'].astype(str)

In [113]:
# drop useless cols
wdf = wdf.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'datetype'])

In [120]:
# store cleaned wdf in ufodata df
ufodata = pd.DataFrame(wdf['year_week'])

In [121]:
# make ufodata cols with cleaned wdf data
ufodata['date_of_sight'] = wdf['dttyp']
ufodata['year'] = wdf['year']
ufodata['month'] = wdf['dttyp'].dt.month
ufodata['week_of_year'] = wdf['weekofyear']
ufodata['state'] = wdf['State']
ufodata['city'] = wdf['City']
ufodata['sight_summary'] = wdf['Summary']
ufodata['ufo_shape'] = wdf['Shape']

In [122]:
# review before export
ufodata.head()

,year_week,date_of_sight,year,month,week_of_year,state,city,sight_summary,ufo_shape
0,2006-52,2006-01-01,2006,1,52,NY,I-80 (unknown city proximity),"Traveling from Brooklyn, NY to Groton, NY on I...",Formation
1,2006-52,2006-01-01,2006,1,52,TX,San Antonio/Laredo (between),Three separate sightins which ocurred in rural...,Triangle
2,2006-1,2006-01-02,2006,1,1,MI,Ann Arbor,"Probable HOAX: 5 ufos, silent black objects, m...",Changing
3,2006-1,2006-01-02,2006,1,1,VA,Fredericksburg,clear sky looked like a star 10:00 am in the m...,Light
4,2006-1,2006-01-04,2006,1,1,PA,Scotland,"Small, white, light in the northeast sky...dis...",Light


In [123]:
# export (set to comment after exporting)
#ufodata.to_csv('../Data/Datasets/ufodata_nb1_clean.csv')